In [ ]:
import pandas as pd
from methods import train_best_xgboost_model, evaluate_best_model, create_timeseries_for_symbol, prepare_regression_data

if __name__ == "__main__":
    # Load and prepare data (with all features and responders)
    df = pd.read_parquet("train.parquet/partition_id=0/part-0.parquet")
    list_of_symbols = df['symbol_id'].unique()

    print(f"\n Total number of symbols (financial instruments) is {len(list_of_symbols)}")
    symbol_id = list_of_symbols[0]

    for symbol_id in list_of_symbols:
        print(f"\nTraining and Evaluating for symbol {symbol_id} ...")
        features, responders, target = create_timeseries_for_symbol(df, symbol_id)
        X, y = prepare_regression_data(features, responders, target)
        
        # Train and evaluate model
        model, X_train, X_test, y_train, y_test = train_best_xgboost_model(X, y)
        evaluate_best_model(model, X_train, X_test, y_train, y_test)


In [ ]:
from methods import make_predictions

# Example usage
if __name__ == "__main__":
    # Load test data
    features = pd.read_parquet("test.parquet/date_id=0/part-0.parquet")
    responders = pd.read_parquet("lags.parquet/date_id=0/part-0.parquet")
    
    # Assuming we have a trained model
    if 'model' in globals():  # Check if model exists
        predictions = make_predictions(model, features, responders)
    else:
        print("\nPlease ensure a trained model exists before running predictions")